### Import libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import torch.nn.functional as F
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler


### Define model

In [2]:
class EEGConformer(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(EEGConformer, self).__init__()

        # CNN
        self.conv1 = nn.Conv2d(1, 40, kernel_size=(1, 25), stride=(1, 1))
        self.conv2 = nn.Conv2d(40, 40, kernel_size=(1, input_dim), stride=(1, 1))
        self.batchnorm = nn.BatchNorm2d(40)

        # Transformer
        self.layernorm1 = nn.LayerNorm(40)
        self.multiheadattention = nn.MultiheadAttention(40, 1)
        self.layernorm2 = nn.LayerNorm(40)

        self.feedworward_block = nn.Sequential(
            nn.Linear(40, 32),
            nn.GELU(),
            nn.Dropout(p=0.1),
            nn.Linear(32, 40)
        )

        # MLP
        self.fc1 = nn.Linear(40, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        # CNN
        x = x.unsqueeze(1).unsqueeze(1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.batchnorm(x)

        # Transformer
        x = x.squeeze()
        x = self.layernorm1(x)
        attn_out = self.multiheadattention(x, x, x)
        x = x + nn.Dropout(0.1)(attn_out[0])
        x = self.layernorm2(x)
        x = self.feedworward_block(x)
        x = nn.Dropout(p=0.1)(x)

        # MLP
        x = self.fc1(x)
        x = F.elu(x)
        x = nn.Dropout(p=0.5)(x)
        x = self.fc2(x)
        x = F.elu(x)
        x = nn.Dropout(p=0.3)(x)
        x = self.fc3(x)
        
        return x

### Load saved trained model

In [3]:
input_dim = 2524  
num_classes = 3   
model = EEGConformer(input_dim, num_classes)

In [4]:
PATH = 'models\Conformer_EEG.pth'
model.load_state_dict(torch.load(PATH, weights_only=True))
model.eval()


EEGConformer(
  (conv1): Conv2d(1, 40, kernel_size=(1, 25), stride=(1, 1))
  (conv2): Conv2d(40, 40, kernel_size=(1, 2524), stride=(1, 1))
  (batchnorm): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layernorm1): LayerNorm((40,), eps=1e-05, elementwise_affine=True)
  (multiheadattention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=40, out_features=40, bias=True)
  )
  (layernorm2): LayerNorm((40,), eps=1e-05, elementwise_affine=True)
  (feedworward_block): Sequential(
    (0): Linear(in_features=40, out_features=32, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=32, out_features=40, bias=True)
  )
  (fc1): Linear(in_features=40, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=3, bias=True)
)

### Load data

In [5]:
eeg_emotions_data = pd.read_csv('emotions.csv')
eeg_emotions_data.head()

,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,...,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b,label
0,4.62,30.3,-356.0,15.6,26.3,1.070,0.411,-15.70,2.06,3.15,...,23.5,20.3,20.3,23.5,-215.0,280.00,-162.00,-162.00,280.00,NEGATIVE
1,28.80,33.1,32.0,25.8,22.8,6.550,1.680,2.88,3.83,-4.82,...,-23.3,-21.8,-21.8,-23.3,182.0,2.57,-31.60,-31.60,2.57,NEUTRAL
2,8.90,29.4,-416.0,16.7,23.7,79.900,3.360,90.20,89.90,2.03,...,462.0,-233.0,-233.0,462.0,-267.0,281.00,-148.00,-148.00,281.00,POSITIVE
3,14.90,31.6,-143.0,19.8,24.3,-0.584,-0.284,8.82,2.30,-1.97,...,299.0,-243.0,-243.0,299.0,132.0,-12.40,9.53,9.53,-12.40,POSITIVE
4,28.30,31.3,45.2,27.3,24.5,34.800,-5.790,3.06,41.40,5.52,...,12.0,38.1,38.1,12.0,119.0,-17.60,23.90,23.90,-17.60,NEUTRAL


### Pre-processing

In [6]:
X = eeg_emotions_data.iloc[0:3,:].drop(['label'], axis=1)
y = eeg_emotions_data['label']

from sklearn.preprocessing import LabelEncoder
labelencoder_emotions = LabelEncoder()
y = labelencoder_emotions.fit_transform(y)

from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)
type(X)
X.shape

(3, 2548)

### Prediction

In [7]:
prediction = model(torch.from_numpy(X).float()).detach().numpy().argmax(axis=1)
prediction = labelencoder_emotions.inverse_transform(prediction)
prediction

array(['NEGATIVE', 'NEUTRAL', 'POSITIVE'], dtype=object)

### Image generation using diffusion model

In [9]:
import torch
from diffusers import StableDiffusionPipeline

model_id = "CompVis/stable-diffusion-v1-4"
device = "cpu"

pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to(device)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, du

### Image generation for each feeling 
- does not run 

In [10]:
for i in range(len(prediction)):
    prompt = 'a photo of an engineering sutdent going through' + prediction[i] + 'feelings'
    image = pipe(prompt)
    imageFile = 'predicted_image_'+str(i)+'.png'
    image.save(imageFile)
    image.show()

  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 